<a href="https://colab.research.google.com/github/wujj0326/business_agglomeration/blob/master/Data/Yelp_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yelp Dataset for RMDS Capstone

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [18]:
import json 
import tarfile
from pandas import json_normalize

In [3]:
with tarfile.open("/content/drive/My Drive/Capstone/yelp_dataset.tar") as tf:
  # tf.extractall()
  members = tf.getmembers()
  b = tf.extractfile(members[5])
  r = tf.extractfile(members[6])
  b1 = b.readlines()
  r1 = r.readlines()

In [4]:
# read as json and filter it to include only Nevada
b_json = []
b_vegas = []
for st in b1:
  b_json.append(json.loads(st))
for b_filter in b_json:
  if b_filter['state']=='NV':
    b_vegas.append(b_filter)

In [5]:
# The number of business in Nevada
len(b_vegas)

39084

In [6]:
# The business ID for the business in Nevada
b_id = []
for business in b_vegas:
  b_id.append(business['business_id'])

In [13]:
# The length of Review data
len(r1)

8021122

In [11]:
# Sample the first 100k review and filter it to only Nevada
r_json = []
r_vegas = []
for st in r1[:100000]:
  r_json.append(json.loads(st))
for r_filter in r_json:
  if r_filter['business_id'] in b_id:
    r_vegas.append(r_filter)

In [12]:
# The length of review after filtering
len(r_vegas)

34079

In [14]:
r_vegas[:10]

[{'business_id': '-MhfebM0QIsKt87iDN-FNw',
  'cool': 0,
  'date': '2015-04-15 05:21:16',
  'funny': 0,
  'review_id': 'xQY8N_XvtGbearJ5X4QryQ',
  'stars': 2.0,
  'text': 'As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It\'s what real estate agents would call "cozy" or "charming" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it\'s not kid friendly at all. Seriously, don\'t bring them.\n* the secur

In [19]:
yelp_vegas = json_normalize(r_vegas)

In [20]:
yelp_vegas.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2.0,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1.0,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
2,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57
3,L2O_INwlrRuoX05KSjc4eg,5vD2kmE25YBrbayKhykNxQ,nlxHRv1zXGT0c0K51q3jDg,5.0,2,0,0,This is definitely my favorite fast food sub s...,2013-05-07 07:25:25
4,lpFIJYpsvDxyph-kPzZ6aA,dsd-KNYKMpx6ma_sRWCSkQ,FNCJpSn0tL9iqoY3JC73qw,5.0,0,0,0,"Awesome office and staff, very professional an...",2017-07-18 18:31:54


In [21]:
# yelp_vegas.to_csv('yelp_vegas.csv')